In [ ]:
# link: https://portal.loft.com.br/valor-do-metro-quadrado-em-sp-por-bairro-especulometro/
# bairro: valor/m2

In [37]:
import pandas as pd

# Ler o arquivo .txt e extrair os dados
with open('media_bairro.txt', 'r') as file:
    lines = file.readlines()

# Inicializar listas para armazenar os dados
bairros = []
valores_por_m2 = []

# Loop pelas linhas do arquivo e extrair os dados relevantes
for line in lines:
    if ':' in line:
        parts = line.split(':')
        bairro = parts[0].strip()
        valor = float(parts[1].replace('R$', '').replace(',', '').replace('.', '').strip())
        bairros.append(bairro)
        valores_por_m2.append(valor)

# Criando Dataframe
data = {'bairro': bairros, 'valor/m2': valores_por_m2}
df_bairro = pd.DataFrame(data)
df_bairro['valor/m2'] = df_bairro['valor/m2'].astype('int64')
df_bairro.head(3)

,bairro,valor/m2
0,Vila Nova Conceição,16839
1,Jardim Europa,16298
2,Vila Olímpia,12667


In [2]:
df_imoveis = pd.read_csv('../Power BI/Geoloc.csv', sep=';')

# Definindo uma função para aplicar a substituição com fallback para valores não no dicionário
def replace_with_fallback(value):
    return dicionario_bairros.get(value, value)

dicionario_bairros = {'MoemaÍndios': 'Moema Índios', 
                      'Bosqueda Saúde': 'Bosque da Saúde', 
                      'FreguesiadoÓ': 'Freguesia do Ó', 
                      'Altode Pinheiros': 'Alto de Pinheiros', 
                      'Altoda Lapa': 'Alto da Lapa', 
                      'JardimÂngela': 'Jardim Ângela', 
                      'Parquedo Carmo': 'Parque do Carmo'}

df_imoveis['Bairro'] = df_imoveis['Bairro'].apply(replace_with_fallback)

df_imoveis.head(3)

,Link_Apto,Endereco,Bairro,Valor_RS,Andares,Tamanho_m2,Valor_RS/m2,Quartos,Suites,Garagem,Banheiros,Varanda,Mobiliado,Portaria,Metro,id,Refdate,Lat,Long
0,https://loft.com.br/imovel/apartamento-17-de-j...,Condomínio Click Morumbi - Rua 17 de Janeiro -...,Vila Andrade,272906,4º andar em diante,40,6822,2,0,1,1,0,Não,24h,100000000,12431,2023-04-17T00:00,-20.341310,-49.197098
1,https://loft.com.br/imovel/apartamento-academi...,Avenida Academia de São Paulo - Itaim Paulista...,Itaim Paulista,245518,3º andar,52,4721,2,1,0,1,0,Não,24h,100000000,12432,2023-04-17T00:00,-23.501765,-46.399609
2,https://loft.com.br/imovel/apartamento-aguiar-...,Condomínio Helbor Art Paulista - Rua Aguiar de...,República,1380000,4º andar em diante,65,21230,2,1,2,2,0,Não,24h,334,12433,2023-04-17T00:00,-23.545335,-46.642257


In [55]:
df_imoveis['Bairro'].unique()

array(['Vila Andrade', 'Itaim Paulista', 'República', 'Santa Cecília',
       'Jardim Paulista', 'Moema Índios', 'Moema Pássaros',
       'Jardim América', 'Bela Vista', 'Paraíso', 'Centro', 'Cursino',
       'Carrão', 'Campo Limpo', 'Morumbi', 'Jardim São Paulo',
       'São Lucas', 'Brás', 'Jaraguá', 'Campos Elísios', 'Higienópolis',
       'Jaguaré', 'Pirituba', 'Campo Belo', 'Ipiranga', 'Tatuapé',
       'Itaim Bibi', 'Limão', 'Tucuruvi', 'Vila Mariana', 'Cidade Ademar',
       'Jabaquara', 'Alto de Pinheiros', 'Sacomã', 'Mandaqui',
       'Rio Pequeno', 'Vila Olímpia', 'Vila Sônia', 'Aclimação',
       'Vila Leopoldina', 'Pinheiros', 'Saúde', 'Campo Grande',
       'Bosque da Saúde', 'Vila Medeiros', 'Cachoeirinha', 'Cambuci',
       'Freguesia do Ó', 'Brooklin', 'Mooca', 'Perdizes', 'Sumaré',
       'Chácara Klabin', 'Santo Amaro', 'Sapopemba', 'Jardim Marajoara',
       'Barra Funda', 'Água Rasa', 'Artur Alvim', 'Vila Maria', 'Belém',
       'Vila Prudente', 'Vila Formosa', 'Lib

In [3]:
print('Quantidade de bairros unicos no DF Imoveis: ', df_imoveis['Bairro'].nunique())
print('Quantidade de bairros unicos no DF bairro: ', df_bairro['bairro'].nunique())

Quantidade de bairros unicos no DF Imoveis:  118
Quantidade de bairros unicos no DF bairro:  57


In [6]:
lista1 = df_imoveis['Bairro'].unique().tolist()
lista2 = df_bairro['bairro'].unique().tolist()

elementos_em_comum = set(lista1) & set(lista2)
len(elementos_em_comum)

57

### Spark -> Transformando os valores necessários

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Inicializando a sessão Spark
spark = SparkSession.builder.appName("ReplaceWithFallback").getOrCreate()

# Exemplo de DataFrame
data = {'A': [1, 2, 3, 4, 5],
        'B': [10, 20, 30, 40, 50]}

df = spark.createDataFrame(pd.DataFrame(data))

# Dicionário de substituição
replace_dict = {2: 200, 4: 400}

# Função para aplicar a substituição com fallback para valores não no dicionário
def replace_with_fallback(value):
    return replace_dict.get(value, value)

# Registrando a função UDF (User-Defined Function)
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

replace_udf = udf(replace_with_fallback, IntegerType())

# Aplicando a UDF à coluna 'A'
df = df.withColumn('A', replace_udf(col('A')))

df.show()

# Encerrando a sessão Spark
spark.stop()

## Analisando Media_Bairro2

In [34]:
df_bairro2 = pd.read_csv('media_bairro2.csv')
df_bairro2 = df_bairro2.rename(columns={'Bairro': 'bairro', 'Valor médio m²': 'valor/m2'})
df_bairro2['valor/m2'] = df_bairro2['valor/m2'].apply(lambda x: x.replace('R$ ', '').replace('.', '').replace(',00', ''))
df_bairro2['valor/m2'] = df_bairro2['valor/m2'].str.replace(',', '').astype('int64')
df_bairro2.head()

,bairro,valor/m2
0,Vila Mariana,10000
1,Morumbi,6380
2,Moema,10910
3,Saúde,8380
4,Ipiranga,7800
